In [ ]:
dbutils.widgets.text("data_execucao", "")
data_execucao = dbutils.widgets.get("data_execucao")

In [ ]:
import requests
from pyspark.sql.functions import lit


In [ ]:
# Extraindo dados da API

# definindo função que faz solicitação de dados para uma API de taxas de câmbio
def extraindo_dados(date, base="BRL"):

    #  construindo URL da API concatenando a data e a moeda base fornecidas como parâmetros
    url = f"https://api.apilayer.com/exchangerates_data/{date}?base={base}"

    # cabeçalhos da solicitação HTTP, incluindo a chave de API.
    headers= {
    "apikey": "inserir código"
    }

    # parâmetros da solicitação, incluindo a moeda base e os símbolos das moedas desejadas para retorno.
    parametros = {
        "base": base,
        "symbols": "USD, GBP, EUR, JPY, CNY, CAD, ZAR, ARS " # Escolhendo as moedas de retorno
        }

    # envia solicitação HTTP GET para a URL da API, passando os cabeçalhos e os parâmetros definidos
    response = requests.request("GET", url, headers=headers, params=parametros)

    # verifica se a resposta da solicitação tem um código de status 200 (indicando uma resposta bem-sucedida)
    if response.status_code != 200:
        raise Exception("Não consegui extrair dados!!!") # se houver erro, exceção é levantada com uma mensagem

    # retorna os dados da resposta da API no formato JSON
    return response.json()

In [ ]:
# função converte os dados de taxas de câmbio no formato JSON para uma lista de tuplas
def dados_para_dataframe(dado_json):
    dados_tupla = [(moeda, float (taxa)) for moeda, taxa in dado_json['rates'].items()]
    return dados_tupla

In [ ]:
def salvar_arquivo_parquet(conversoes_extraidas):
    ano, mes, dia = conversoes_extraidas['date'].split('-')
    caminho_completo = f"dbfs:/databricks-results/bronze/{ano}/{mes}/{dia}"
    response = dados_para_dataframe(conversoes_extraidas)
    df_conversoes = spark.createDataFrame(response, schema=['moeda', 'taxa'])
    df_conversoes = df_conversoes.withColumn("data", lit(f"{ano}-{mes}-{dia}"))
    
    df_conversoes.write.format("parquet")\
                .mode("overwrite")\
                .save(caminho_completo)

    print(f"Os arquivos foram salvos em {caminho_completo}")

In [ ]:
# chamando a função para obter os dados de cotações na data de interesse
cotacoes = extraindo_dados(data_execucao)
salvar_arquivo_parquet(cotacoes)

Os arquivos foram salvos em dbfs:/databricks-results/bronze/2024/03/01


In [ ]:
display(dbutils.fs.ls("dbfs:/databricks-results/"))

path,name,size,modificationTime
dbfs:/databricks-results/bronze/,bronze/,0,1711856759000
